In [1]:
import tensorflow as tf
import numpy as np

In [17]:
#Parameters
learning_rate = 0.001

#Network Parameters
movie_num = 3883
score_num = 5 
batch_size = 512
hidden_num = 500

In [21]:
ratings = tf.placeholder(tf.float32, shape=[None, movie_num, score_num])
in_mask = tf.placeholder(tf.float32, shape=[None, movie_num])
out_mask = tf.placeholder(tf.float32, shape=[None, movie_num])

In [22]:
def CF_NADE(ratings, in_mask, out_mask):
    bias = {'l1':tf.Variable(tf.random_normal(shape=[hidden_num])),
           'l2':tf.Variable(tf.random_normal(shape=[movie_num, score_num]))}
    weight = {'l1':tf.Variable(tf.random_normal(shape=[movie_num, score_num, hidden_num])),
             'l2':tf.Variable(tf.random_normal(shape=[hidden_num, movie_num, score_num]))}
    # dim(h) = batch_size * hidden_num
    h = tf.tanh(tf.add(bias['l1'], tf.tensordot(ratings
                                                * in_mask[:,:,np.newaxis], weight['l1'], axes=[[1,2], [0,1]])))
    #dim(output) = batch_size * movie_num * socre_num
    output = tf.add(bias['l2'], tf.tensordot(h, weight['l2'], axes=[[1], [0]]) * out_mask[:,:,np.newaxis])
    return output

In [24]:
output = CF_NADE(ratings, in_mask, out_mask)
#dim(scores_tensor) = batch_size * movie_num * score_num
scores_tensor = np.concatenate([np.ones((batch_size, movie_num, 1)) * i for i in range(1, 6)], axis=2)
#dim(batch_socres) = batch_size * movie_num
pred_scores = tf.reduce_sum(scores_tensor * tf.nn.softmax(output, axis=2), axis=2)
true_scores = tf.argmax((ratings * out_mask[:,:,np.newaxis]), axis=2) + 1
loss_op = tf.losses.mean_squared_error(true_scores, pred_scores)
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss_op)
init = tf.global_variables_initializer()

In [16]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epoches):
        for batch in train_loop_stream.get_epoch_iterator(): # to do
            ratings, in_mask, out_mask = batch
            sess.run(train_op, feed_dict={ratings:ratings, in_mask:in_mask, out_mask:out_mask})
        loss = sess.run(loss_op, feed_dict={ratings:ratings, in_mask:inmask, out_mask:outmask})
        print ('epoch:', epoch, 'loss:', loss)

NameError: name 'epoches' is not defined